In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import scipy.stats as stats

In [ ]:
df = pd.read_csv('/root/bryton/github/cv_research/bryton/ml_sealice/2016.csv')

In [ ]:
num = df.loc[df['avgAdultFemaleLice'] < 0.5, 'avgAdultFemaleLice']
#num = df.loc[df['avgMobileLice'] < 1.5, 'avgMobileLice']
#num = df.loc[df['avgStationaryLice'] < 0.5, 'avgStationaryLice']

dist = dist=stats.distributions.beta
res = num
#res = np.log(1 + num)

fig = sm.qqplot(res, dist=dist, fit=True, line='45')

plt.show()

In [ ]:
dist= dist=stats.distributions.expon

In [ ]:
print(df.iloc[0])

plt.hist(df.loc[df['avgAdultFemaleLice'] < 1.5, 'avgAdultFemaleLice'], bins='auto', density=True)
plt.show()

plt.hist(df.loc[df['avgMobileLice'] < 5, 'avgMobileLice'], bins='auto', density=True)
plt.show()

plt.hist(df.loc[df['avgStationaryLice'] < 3, 'avgStationaryLice'], bins='auto', density=True)
plt.show()

print(np.mean(df['avgAdultFemaleLice']))
print(np.median(df['avgAdultFemaleLice']))

print(np.mean(df['avgMobileLice']))
print(np.median(df['avgMobileLice']))

print(np.mean(df['avgStationaryLice']))
print(np.median(df['avgStationaryLice']))

for i in range(0, 100, 1):
    p1 = np.percentile(df['avgAdultFemaleLice'], i) # return 50th percentile, e.g median.
    p2 = np.percentile(df['avgMobileLice'], i) # return 50th percentile, e.g median.
    p3 = np.percentile(df['avgStationaryLice'], i) # return 50th percentile, e.g median.
    print('%0.2f: %0.2f, %0.2f, %0.2f' % (i, p1, p2, p3))

In [ ]:
my_df = df.loc[:,'avgAdultFemaleLice':'seaTemperature']

new_df = my_df.copy()
norm_df = my_df.copy()

for x in my_df.columns:
    my_mean = my_df[x].median()
    my_std = my_df[x].std()
    my_iqr = stats.iqr(my_df[x])
    
    my_row = my_df[x].copy()
    new_df[x] = my_row / my_iqr
    norm_df[x] = (my_row - my_mean) / my_std

In [ ]:
#good_indices_1 = (my_df == 0).any(axis=1) == False
good_indices_2 = my_df['avgAdultFemaleLice'] < 1.5
good_indices_3 = my_df['avgMobileLice'] < 5
good_indices_4 = my_df['avgStationaryLice'] < 3

good_indices = good_indices_2 & good_indices_3 & good_indices_4

print(np.sum(good_indices) / len(good_indices))

my_df_X = new_df.filter(['avgAdultFemaleLice', 'avgMobileLice', 'seaTemperature'], axis=1)
my_df_Y = new_df['avgStationaryLice'][good_indices]

my_df_X = my_df_X.loc[good_indices, :]

my_df_X = np.log(1 + my_df_X)
my_df_Y = my_df_Y
#my_df_Y = np.log(1 + my_df_Y)

my_df_X = np.hstack((my_df_X, my_df_X ** 2, np.log(1 + my_df_X)))

myX = sm.add_constant(my_df_X)
myY = np.log(1 + my_df_Y)

model = sm.OLS(myY, myX).fit()

res = model.resid
fig = sm.qqplot(res, dist=stats.distributions.norm, fit=True, line='45')

plt.show()

print(model.summary())

In [ ]:
plt.scatter(my_df_X[:,0], my_df_Y)
plt.show()

In [ ]:
predY = model.predict(myX)
error = predY - myY

avg_error = np.median(np.abs(error))

print(avg_error)

plt.scatter(myY, predY)
plt.xlabel('Actual stationary')
plt.ylabel('Predicted stationary')
plt.show()

In [ ]:
res_X = predY
res_Y = predY - myY

res_myX = sm.add_constant(res_X)
res_myY = res_Y

res_model = sm.OLS(res_myY, res_myX).fit_regularized(L1_wt=0)
res_predY = res_model.predict(res_myX)

print(res_model.summary())

plt.scatter(res_X, res_Y)
plt.plot(res_X, res_predY, color='red')
plt.show()

In [ ]:
plt.scatter(np.exp(myY), np.exp(predY))
plt.xlabel('Actual stationary')
plt.ylabel('Predicted stationary')
plt.show()

error = np.exp(predY) - np.exp(myY)

avg_error = np.median(np.abs(error))

print(avg_error)

In [ ]:
# Trying a different model that is time-series based

In [ ]:
subset = my_df.loc[df['localityNo'] == 11756]

print(subset)

plt.plot(subset['avgAdultFemaleLice'])
plt.show()

In [ ]:
print(subset.shape)

subset_v = subset.values
print(subset_v.shape)

diff = subset_v[1:21] - subset_v[0:20]

diff_myX = np.hstack((subset_v[1:21,0:1], subset_v[0:20,:]))
diff_myY = subset_v[1:21, 2]

diff_model = sm.OLS(diff_myY, diff_myX).fit()
diff_predY = diff_model.predict(diff_myX)

print(diff_model.summary())

plt.scatter(diff_myY, diff_predY)
plt.show()

In [ ]:
localityNumbers = np.unique(df['localityNo'])

all_diff_myX = None
all_diff_myY = None

for localityNo in localityNumbers:
    subset = my_df.loc[df['localityNo'] == localityNo]
    subset_v = subset.values
    num_obs = subset_v.shape[0]

    diff = subset_v[1:num_obs] - subset_v[0:(num_obs - 1)]
    
    diff_myX = np.hstack((subset_v[1:num_obs,0:2], subset_v[0:(num_obs - 1),0:3]))
    diff_myY = subset_v[1:num_obs, 2]
    
    if all_diff_myX is None:
        all_diff_myX = diff_myX
        all_diff_myY = diff_myY
    else:
        all_diff_myX = np.vstack((all_diff_myX, diff_myX))
        all_diff_myY = np.concatenate((all_diff_myY, diff_myY))
        
print('done')
print(all_diff_myX.shape)
print(all_diff_myY.shape)

In [ ]:
all_diff_myX

In [ ]:
good_indices_2 = all_diff_myX[:,0] < 1.5
good_indices_3 = all_diff_myX[:,1] < 5
good_indices_4 = all_diff_myX[:,2] < 1.5
good_indices_5 = all_diff_myX[:,3] < 5
good_indices_6 = all_diff_myX[:,4] < 3
good_indices_7 = all_diff_myY < 3

good_indices = good_indices_2 & good_indices_3 & good_indices_4 & good_indices_5 & good_indices_6 & good_indices_7

print(np.sum(good_indices) / len(good_indices))

all_filtered_myX = all_diff_myX[good_indices, :]
all_filtered_myY = all_diff_myY[good_indices]

all_myX = sm.add_constant(np.hstack((all_filtered_myX, np.log(1 + all_filtered_myX))))
all_myY = np.log(1+all_filtered_myY)

all_diff_model = sm.OLS(all_myY, all_myX).fit()
all_predY = all_diff_model.predict(all_myX)

print(all_diff_model.summary())

plt.scatter(all_myY, all_predY)
plt.show()

In [ ]:
final_predY = np.exp(all_predY) - 1
final_myY = np.exp(all_myY) - 1

error = np.median(np.abs(final_predY - final_myY))

print(np.median(final_myY))

print(error)